## MongoDB 連線

In [94]:
import re
from pymongo import MongoClient 

#預設就是自己
client = MongoClient('10.120.28.12',27017)
database = client['test']
collection =database['news2']

## Search + Jieba 斷詞

In [95]:
import re
import jieba
import time
tic = time.clock()
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫

#給關鍵字----------------------------------------
tag1 = "柯文哲"
tag2 = "單車"
#-----------------------------------------------

date=[]
title=[]
content =[]
tfidf = []
#把資料庫東西抓出來
for post in collection.find(
    {"$and":[                   
            {"tfidf":tag1},
            {"tfidf":tag2},
            #{"date":{"$regex":"2016"}},
            ]},{"_id":0}): 
    summary = post['content']
    content.append(' '.join(jieba.cut(summary)))
    title.append(post['title'])
    date.append(post['date'])
    
    tfidf.append(post['tfidf'])

#文章數
print "文章數:",len(title)
toc = time.clock()
print "執行時間:",(toc - tic),"秒"

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.558 seconds.
Prefix dict has been built succesfully.


文章數: 52
執行時間: 2.55575541206 秒


## TF_IDF 計算 

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import time
tic = time.clock()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)  # titile 放文本
weight = X.toarray()

#特徵值總共有多少個
print "Feature_num",len(vectorizer.get_feature_names())

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()

toc = time.clock()
print "執行時間:",(toc - tic),"秒"

Feature_num 4378
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
執行時間: 0.0485392323371 秒


## K-means 模型訓練

In [92]:
import time
tic = time.clock()
from sklearn import cluster
c = cluster.KMeans(n_clusters=2)
k_data = c.fit_predict(weight)  # weight 直接拿來用 或放 X 有多算一層TF_IDF 分群效果會好一點

toc = time.clock()
print "執行時間:",(toc - tic),"秒"

執行時間: 0.0518608664752 秒


## 看分群結果

In [93]:
for idx, grp in enumerate(k_data):
    if grp == 0:
        print title[idx],date[idx]
print "-----------------------------------------------------------------------------------------------------------------------"
for idx, grp in enumerate(k_data):
    if grp == 1:
        print title[idx],date[idx]

柯文哲戰駒,今午1元起標雙城雙塔2車值37萬,標金捐作公益 20160304
戰駒飆74萬,柯允陪騎2分鐘 20160306
戰駒車衣義賣450萬,柯：陪騎感謝買主 20160316
義賣戰駒車衣,柯捐335萬車衣面交教練,2單車買家傳是捷安特 20160327
雙塔戰車與車衣 柯P義賣 20160304
-----------------------------------------------------------------------------------------------------------------------
淡化藍綠,柯朱會：合作沒問題互虧大學把妹,談笑啟動雙北論壇 20141204
周六拼「一日雙塔」,柯將缺席228紀念 20160225
太強了,太痛苦了,28小時騎完雙塔,柯：絕不再幹 20160229
柯P鐵騎長征 變輔選之旅 20151214
柯P鐵騎長征 變輔選之旅 20151214
完成19.5小時北高長征 柯P早起探扁 網友讚狂伯 20160111
柯P「點亮板橋螢光夜騎」 笑稱「考察板橋路面」 20160115
流感升溫 柯挨批 只想到一日雙塔 20160226
雙城到雙塔 看柯P的單車政治學 20160228
雙城到雙塔 看柯P的單車政治學 20160228
28.5小時 520公里長征 柯P：汗水代替淚水 以寬容撫228傷痕 20160229
28.5小時 520公里長征 柯P：汗水代替淚水 以寬容撫228傷痕 20160229
揭密柯文哲的單車夢　徐欣瑩：從「雙塔」變「雙城」 20151213
非典型輔選！柯文哲騎到新竹　與徐欣瑩一同踩單車 20151213
柯P將挑戰單車一日北高　網友：進擊的阿伯沒有極限！ 20160105
航向不可知未來！　柯P將挑戰一日雙塔：我就像哥倫布 20160107
脫衣給失溫山友穿　柯P玉山上跳4小時：醫生就是要救人 20160112
柯P一日雙城造假？　車友張惠忠：當然是真騎還沒交管 20160114
「一日雙城」狂飆380km是假的？　柯P：這有什麼好造假 20160114
回新竹拜城隍 柯讚林智堅「我的分身」 20141124
柯單車助選 長征15小時240公里9縣市 20151214
〈中部〉柯P鐵騎助綠將 藍將拉攏新住民 20160110
柯文哲單車行 直呼屁股好痛  

## 將訓練好的模型存起來
- 共三份檔案，要放在同一個資料夾中
- 官方原文 http://scikit-learn.org/stable/modules/model_persistence.html

In the specific case of the scikit, it may be more interesting to use joblib’s replacement of pickle (joblib.dump & joblib.load), which is more efficient on objects that carry large numpy arrays internally as is often the case for fitted scikit-learn estimators, but can only pickle to the disk and not to a string:

from sklearn.externals import joblib
joblib.dump(clf, 'filename.pkl') 
Later you can load back the pickled model (possibly in another Python process) with:

clf = joblib.load('filename.pkl') 
Note joblib.dump returns a list of filenames. Each individual numpy array contained in the clf object is serialized as a separate file on the filesystem. All files are required in the same folder when reloading the model with joblib.load.

In [72]:
import time
tic = time.clock()

from sklearn.externals import joblib
joblib.dump(c, 'filename.pkl') 

toc = time.clock()
print "執行時間:",(toc - tic),"秒"

執行時間: 0.0193022490685 秒


## 載入 訓練好的模型並看結果

In [75]:
import time
tic = time.clock()

c = joblib.load('filename.pkl') 
k_data = c.fit_predict(weight)
for idx, grp in enumerate(k_data):
    if grp == 0:
        print title[idx],date[idx]
print "---------------------------------"
for idx, grp in enumerate(k_data):
    if grp == 1:
        print title[idx],date[idx]
        
toc = time.clock()
print "執行時間:",(toc - tic),"秒"

騎鐵馬環台北,柯P挺8選將記仇MG149,痛批費鴻泰「道德有問題」 20160103
玉山救人是假的?被救者「若無柯P已做頭七」 20160116
戰駒飆74萬,柯允陪騎2分鐘 20160306
義賣戰駒車衣,柯捐335萬車衣面交教練,2單車買家傳是捷安特 20160327
北高騎跡輔選 柯拚21時380公里 20160107
單車北高一日長征380公里 柯P輔選實境秀 選將搭便車 20160110
雙城到雙塔 看柯P的單車政治學 20160228
雙城到雙塔 看柯P的單車政治學 20160228
雙塔戰車與車衣 柯P義賣 20160304
蔡英文續任黨主席 柯文哲：做做看、不行再講 20160304
柯P戰駒戰衣義賣 450萬結標 20160316
揭密柯文哲的單車夢　徐欣瑩：從「雙塔」變「雙城」 20151213
讓新力量進國會改革！　林昶佐揪柯P化身Rocker騎單車 20160102
上任週年施政滿意度69.3%　柯文哲：低於65%就收斂點 20160102
一日雙塔將沿路輔選？柯文哲：坐上去騎就好，安排什麼 20160105
嗡嗡嗡到屏東！柯P雙塔任務　將從富貴角騎單車到鵝鑾鼻 20160105
騎到屏東會有候選人靠行？　柯文哲：那他要騎車跟旁邊 20160106
21小時、380公里！柯文哲9日凌晨開始挑戰一日雙城任務 20160108
衝破藍綠封鎖線！柯文哲單車挑戰一日雙城　已橫越濁水溪 20160109
雖延遲20分鐘出發　柯文哲單車雙城挑戰趕上既定行程 20160109
民眾注意！柯Ｐ4:30準備「野生現雙城」：來不及夢就醒了 20160109
鐵人柯P掀總統級旋風　220萬人爭睹北高380公里狂飆！ 20160110
不累？57歲柯P北高狂騎380公里　隔天早上9點餐廳現身！ 20160110
結束380公里北高長征　柯文哲神恢復7點起床：睡一晚上就好了 20160110
歐吉桑柯P一日北高　藤井樹：搭車回家投票還嫌麻煩？ 20160110
歐吉桑柯P的「不妥協夢」：2014年時，改變…也成真 20160110
高雄私人行程探阿扁1小時　柯文哲：會好嘛抹完全 20160111
柯文哲完成一日北高　潘孟安：我在屏東等你喔 20160111
脫衣給失溫山友穿　柯P玉山上跳4小時：醫生就是要救人 20160112
腳踏車日比犯罪少年重要？　李茂生批柯讓